In [ ]:
!pip install -q google-adk

In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set

# Recipe Helper Agent

Build an agent that helps users cook. Your agent will have these tools:

1. **search_recipes** — Find recipes that use given ingredients
2. **convert_units** — Convert between cooking measurement units
3. **suggest_substitution** — Suggest ingredient substitutions for dietary needs or missing items

Tools return hardcoded/fake data — the goal is designing the **interface** between the model and your code.

## Setup

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.genai import types

## TODO 1: Define your tools

Just write plain Python functions. The ADK infers the schema from:
- **Type hints** → parameter types
- **Docstring** → tool description

The first tool is done for you as an example. Implement the remaining two.

In [ ]:
# Example tool (complete)

def search_recipes(ingredients: str) -> dict:
    """Search for recipes that can be made with the given ingredients.

    Args:
        ingredients: Comma-separated list of ingredients (e.g. "chicken, rice, garlic").

    Returns:
        dict: status and matching recipes or error message.
    """
    return {
        "status": "success",
        "report": (
            f"Found 3 recipes using {ingredients}: "
            f"1) Garlic Chicken Stir-Fry (30 min, easy). "
            f"2) Chicken Fried Rice (25 min, easy). "
            f"3) One-Pot Chicken & Rice (45 min, medium)."
        ),
    }


def convert_units(amount: float, from_unit: str, to_unit: str) -> dict:
    """Convert between cooking measurement units.

    Args:
        amount: The quantity to convert.
        from_unit: Source unit (e.g. "cups", "tablespoons", "grams").
        to_unit: Target unit (e.g. "ml", "teaspoons", "ounces").

    Returns:
        dict: status and converted measurement or error message.
    """
    # TODO: implement — return a dict with "status" and "report" keys
    # Hint: use hardcoded conversion factors (e.g. 1 cup = 236.6 ml)
    pass


def suggest_substitution(ingredient: str, reason: str) -> dict:
    """Suggest a substitution for an ingredient.

    Args:
        ingredient: The ingredient to replace (e.g. "butter").
        reason: Why a substitution is needed (e.g. "vegan", "allergy", "out of stock").

    Returns:
        dict: status and substitution suggestions or error message.
    """
    # TODO: implement — return a dict with "status" and "report" keys
    # Hint: return 1-2 substitution options with usage notes
    pass

## TODO 2: Define your agent

In [ ]:
agent = Agent(
    model="gemini-3-flash-preview",
    name="recipe_helper",
    # TODO: Write an instruction that gives the agent a friendly chef persona.
    # Hint: describe what the agent can help with, dietary awareness, tone, etc.
    instruction="""You are a helpful recipe assistant.

Replace this with your agent's persona and guidelines.
""",
    tools=[search_recipes, convert_units, suggest_substitution],
)

## Run your agent

Once you've filled in the TODOs above, run this cell to test your agent!

In [ ]:
runner = InMemoryRunner(agent=agent, app_name="recipe_helper")
runner.auto_create_session = True
user_content = types.Content(role="user", parts=[types.Part.from_text(text="I have chicken, garlic, and rice. What can I make? Also, I need to convert 2 cups of rice to grams.")])

final_text = ""
async for event in runner.run_async(user_id="user1", session_id="session1", new_message=user_content):
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.function_call:
                print(f"🔧 Tool call: {part.function_call.name}({dict(part.function_call.args)})")
            if part.function_response:
                print(f"  ↳ Response: {part.function_response.name} returned")
            if part.text:
                final_text = part.text

print(f"\n{'='*60}\nFinal response:\n{'='*60}\n{final_text}")

## Tips

- **Docstrings matter** — the ADK uses them as tool descriptions. Be specific about what each parameter means.
- **Fake data is fine** — the goal is designing the agent's interface, not building a real recipe database.
- **Test edge cases** — what happens when a user asks for a recipe with unusual ingredients?

### Stretch Goals
- Add a `get_nutrition_info` tool that returns calorie/macro estimates for a dish
- Add a `scale_recipe` tool that adjusts ingredient quantities for a different serving size
- Handle dietary restrictions (e.g. "find me a vegan version of this recipe")